In [1]:
import numpy as np
import pandas as pd
import os
import re
import collections
import time
import nltk
import string
import os
from os.path import exists
from nltk.corpus import stopwords
# Cache stopword set so access is faster
stop_words = set(stopwords.words('english'))
translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))

**Prepare Data for Analysis (Run Once)**

In [2]:
# data = '../../OutputPersons/PersonsBreakdown'
# output = '../../Data/CorporaData'

**Grab all of the Necessary File Tuples**

In [3]:
# files = []
# for file in os.listdir(data):
#     if file != '.DS_Store' and file != '.ipynb_checkpoints' and file != 'README.txt':
#         for year in os.listdir(output):
#             if year != '.DS_Store':
#                 for o_file in os.listdir(os.path.join(output, year)):
#                     if o_file != '.ipynb_checkpoints':
#                         if '{}_{}'.format(o_file.split('.')[0], o_file.split('.')[2]) == file.split('.')[0]:
#                             o_filedf = pd.read_csv(os.path.join(output, os.path.join(year, o_file)))
#                             o_filedf.columns = ['URL', 'Title', 'RawText']
#                             files.append((os.path.join(data, file), pd.read_csv(os.path.join(data, file)), o_filedf))

**Add Associated Raw Text for each show to our current Dataframes**

In [4]:
# dataframes = []
# for tpl in files:
#     p_df = tpl[1]
#     o_df = tpl[2]
#     merged_df = pd.merge(p_df, o_df, on='Title')
#     merged_df.drop_duplicates(inplace=True)
#     merged_df.reset_index(drop=True, inplace=True)
#     dataframes.append((tpl[0], merged_df))

**Preprocess the Raw Text (remove 'Topics' as they mess with the frequency of mention we are trying to calculate)**

Save Processed Dataframes to CSV so we dont have to re-run this code (time intensive)

In [5]:
# s = time.time()
# for tpl in dataframes:
#     if exists('../../OutputPersons/PersonsData/{}'.format(tpl[0].split('/')[3])):
#         continue
#     else:
#         tpl[1]['RawText_preprocessed'] = tpl[1]['RawText'].apply(
#             lambda text: str(text).split('TOPICS: TOPIC FREQUENCY ')[0])
#         tpl[1]['RawText_preprocessed'] = tpl[1]['RawText'].apply(lambda text: str(text)[str(text).find('[[TITLE.END]] ') + len('[[TITLE.END]] '):])   
#         tpl[1]['RawText_preprocessed'] = tpl[1]['RawText_preprocessed'].apply(lambda text: ' '.join(re.sub('<[^<]+?>', '', word) for word in text.split()))   
#         tpl[1]['RawText_preprocessed'] = tpl[1]['RawText_preprocessed'].apply(lambda text: text.translate(translator))
#         del tpl[1]['RawText']
#         del tpl[1]['URL']
#         tpl[1].to_csv('../../OutputPersons/PersonsData/{}'.format(tpl[0].split('/')[3]), index=False)
# e = time.time()
# print('Time taken to preprocess: {} seconds'.format(e-s))

**Reference Pre-Prepared Data for Bigrams Task**

In [2]:
persons_data = '../../OutputPersons/PersonsData'
# persons_data = '../../OutputPersons/PersonsBigramsRaw'
# persons_data = '../../OutputPersons/PersonsBigramsFormatted'

In [3]:
persons_files = []
for file in os.listdir(persons_data):
    if file != '.DS_Store' and file != '.ipynb_checkpoints':
        persons_files.append(os.path.join(persons_data, file))

In [4]:
len(persons_files)

53

**Find Frequency of Mention of First and Last Name as Unigrams Associated with a Bigram**

In [26]:
for persons_f in persons_files:
    start = time.time()
    df = pd.read_csv(persons_f)
    if exists('../../OutputPersons/PersonsBigramsRaw/{}'.format(persons_f.split('/')[3])):
        continue
    else:
        for i in range(len(df.index)): 
            res_list = {}
            for name in df.loc[i, 'Bigrams of Interest'].split(', '):
                res_list[name.split(': ')[0]] = {}
                for k,v in res_list.items():
                    firstname = k.split()[0]
                    lastname = k.split()[1]
                    res_list[k] = {firstname: str(df.loc[i, 'RawText_preprocessed']).lower().count(' {} '.format(firstname)), lastname: str(df.loc[i, 'RawText_preprocessed']).lower().count(' {} '.format(lastname))}
            # print(', '.join('{} | {}'.format(k, v) for k,v in res_list.items()))
            df.loc[i, 'References'] = ', '.join('{} | {}'.format(k, v) for k,v in res_list.items())
        df.to_csv('../../OutputPersons/PersonsBigramsRaw/{}'.format(persons_f.split('/')[3]), index=False)
        end = time.time()
        print('Time Taken to run file {}: {} seconds'.format(persons_f, end-start))
    

Time Taken to run file ../OutputPersons/PersonsData/FBC_2017.csv: 331.026734828949 seconds
Time Taken to run file ../OutputPersons/PersonsData/CNN_2014.csv: 368.3518850803375 seconds
Time Taken to run file ../OutputPersons/PersonsData/CNN_2016.csv: 536.1193289756775 seconds
Time Taken to run file ../OutputPersons/PersonsData/FBC_2015.csv: 183.5704469680786 seconds
Time Taken to run file ../OutputPersons/PersonsData/FBC_2014.csv: 53.1226909160614 seconds
Time Taken to run file ../OutputPersons/PersonsData/CNN_2017.csv: 357.4926872253418 seconds


**Re-format the files according to preferred format**

In [32]:
for file in persons_files:
    start = time.time()
    df = pd.read_csv(file)
    if exists('../../OutputPersons/PersonsBigramsFormatted/{}'.format(persons_f.split('/')[3])):
        continue
    else:
        tuples_list = []
        output_df = pd.DataFrame()
        for i in range(len(df.index)):
            show_title = str(df.loc[i, 'Title'])
            references_list = df.loc[i, 'References'].split('}')
            bigrams_list = df.loc[i, 'Bigrams of Interest'].split(',')
            for item in references_list:
                if item != '':
                    unigram = re.sub("'", '', item.split('|')[1].replace('{', '').lstrip())
                    main_person = bigrams_list[references_list.index(item)]
                    tuples_list.append((show_title, main_person, unigram))
        output_df = pd.DataFrame(tuples_list, columns=['Title', 'Unigram', 'Bigrams'])
        output_df.to_csv('../../OutputPersons/PersonsBigramsFormatted/{}'.format(file.split('/')[3]), index=False)
        end = time.time()
        print('Time Taken to format {}: {} seconds'.format('../../OutputPersons/PersonsBigramsFormatted/{}'.format(file.split('/')[3]), end-start))

Time Taken to format ../OutputPersons/PersonsBigramsFormatted/FBC_2019.csv: 2.361157178878784 seconds
Time Taken to format ../OutputPersons/PersonsBigramsFormatted/FBC_2018.csv: 1.4864141941070557 seconds
Time Taken to format ../OutputPersons/PersonsBigramsFormatted/CNN_2019.csv: 3.471096992492676 seconds
Time Taken to format ../OutputPersons/PersonsBigramsFormatted/CNN_2018.csv: 3.2041821479797363 seconds
Time Taken to format ../OutputPersons/PersonsBigramsFormatted/CNN_2020.csv: 3.5264902114868164 seconds
Time Taken to format ../OutputPersons/PersonsBigramsFormatted/FBC_2020.csv: 1.8303258419036865 seconds
Time Taken to format ../OutputPersons/PersonsBigramsFormatted/Bloomberg_2015.csv: 1.8072009086608887 seconds
Time Taken to format ../OutputPersons/PersonsBigramsFormatted/FOXNEWS_2016.csv: 2.7688241004943848 seconds
Time Taken to format ../OutputPersons/PersonsBigramsFormatted/CNBC_2016.csv: 1.2873871326446533 seconds
Time Taken to format ../OutputPersons/PersonsBigramsFormatted/FO

**Add Unigram Specific Attribute Info (from wiki_bigrams)**

In [5]:
wiki = pd.read_excel('../../Data/Other/wiki_bigrams.xlsx')
wiki['Info1'] = wiki['Info1'].apply(lambda info: str(info))

In [48]:
attributes = ['Male', 'Female', 'Politician', 'American', 'Foreign', 'Republican', 'Democratic', 'Chief Executive Officer', 
              'Chief Financial Officer', 'Hedge Fund Manager', 'Investor', 'Billionaire', 'Lawyer', 'Television Reporters', 'Television Hosts']

for file in persons_files:
    start = time.time()
    df = pd.read_csv(file)
    # Add something like 'if 'Male' in df.columns: continue'
    for i in range(len(df.index)):
        if str(df.loc[i, 'Unigram']) != 'No Hits':
            person = df.loc[i, 'Unigram'].split(':')[0].lstrip()
            person_index = wiki.loc[wiki['Name'] == person.rstrip()].index[0] # Grab index of person in wiki_bigrams
            for att in attributes:
                if att == 'Female':
                    if att.lower() in wiki.loc[person_index, 'Info1'].lower() or 'women' in wiki.loc[person_index, 'Info1'].lower():
                        df.loc[i, att] = int(1)
                    else:
                        df.loc[i, att] = int(0)
                elif att == 'Male':
                    if 'female' not in wiki.loc[person_index, 'Info1'].lower() and 'women' not in wiki.loc[person_index, 'Info1'].lower():
                        df.loc[i, att] = int(1)
                    else:
                        df.loc[i, att] = int(0)
                elif att == 'Foreign':
                    if 'american' not in wiki.loc[person_index, 'Info1'].lower():
                        df.loc[i, att] = int(1)
                    else:
                        df.loc[i, att] = int(0)
                else:
                    if att.lower() in wiki.loc[person_index, 'Info1'].lower():
                        df.loc[i, att] = int(1)
                    else:
                        df.loc[i, att] = int(0)
        else:
            continue
    df.to_csv(file, index=False)  
    end = time.time()
    print('File {} done: {} seconds'.format(file, end-start))

File ../OutputPersons/PersonsBigramsFormatted/FBC_2019.csv done: 212.97288012504578 seconds
File ../OutputPersons/PersonsBigramsFormatted/FBC_2018.csv done: 195.28288292884827 seconds
File ../OutputPersons/PersonsBigramsFormatted/CNN_2019.csv done: 494.01712799072266 seconds
File ../OutputPersons/PersonsBigramsFormatted/CNN_2018.csv done: 468.3604431152344 seconds
File ../OutputPersons/PersonsBigramsFormatted/CNN_2020.csv done: 401.87814712524414 seconds
File ../OutputPersons/PersonsBigramsFormatted/FBC_2020.csv done: 173.7966079711914 seconds
File ../OutputPersons/PersonsBigramsFormatted/Bloomberg_2015.csv done: 169.29652190208435 seconds
File ../OutputPersons/PersonsBigramsFormatted/FOXNEWS_2016.csv done: 426.3785388469696 seconds
File ../OutputPersons/PersonsBigramsFormatted/CNBC_2016.csv done: 94.63657307624817 seconds
File ../OutputPersons/PersonsBigramsFormatted/FOXNEWS_2017.csv done: 460.69200325012207 seconds
File ../OutputPersons/PersonsBigramsFormatted/CNBC_2017.csv done: 93.

In [7]:
attributes = ['Fox News People', 'Fox Business', 'CNBC People', 'Bloomberg People', 'Fox Business', 'MSNBC People', 'CNN People']

for file in persons_files:
    start = time.time()
    df = pd.read_csv(file)
    # Add something like 'if 'Male' in df.columns: continue'
    for i in range(len(df.index)):
        count = 0
        if str(df.loc[i, 'Unigram']) != 'No Hits':
            person = df.loc[i, 'Unigram'].split(':')[0].lstrip()
            person_index = wiki.loc[wiki['Name'] == person.rstrip()].index[0] # Grab index of person in wiki_bigrams
            for att in attributes:
                if att.lower() in wiki.loc[person_index, 'Info1'].lower():
                    count += 1
            if count > 0:
                df.loc[i, 'Television Person'] = int(1)
            else:
                df.loc[i, 'Television Person'] = int(0)
        else:
            continue
    df.to_csv(file, index=False)  
    end = time.time()
    print('File {} done: {} seconds'.format(file, end-start))

File ../OutputPersons/PersonsBigramsFormatted/FBC_2019.csv done: 80.43328213691711 seconds
File ../OutputPersons/PersonsBigramsFormatted/FBC_2018.csv done: 74.65979313850403 seconds
File ../OutputPersons/PersonsBigramsFormatted/CNN_2019.csv done: 177.40051984786987 seconds
File ../OutputPersons/PersonsBigramsFormatted/CNN_2018.csv done: 176.81460905075073 seconds
File ../OutputPersons/PersonsBigramsFormatted/CNN_2020.csv done: 157.8250036239624 seconds
File ../OutputPersons/PersonsBigramsFormatted/FBC_2020.csv done: 78.18749594688416 seconds
File ../OutputPersons/PersonsBigramsFormatted/Bloomberg_2015.csv done: 78.11636090278625 seconds
File ../OutputPersons/PersonsBigramsFormatted/FOXNEWS_2016.csv done: 167.5404818058014 seconds
File ../OutputPersons/PersonsBigramsFormatted/CNBC_2016.csv done: 49.36315107345581 seconds
File ../OutputPersons/PersonsBigramsFormatted/FOXNEWS_2017.csv done: 170.5167601108551 seconds
File ../OutputPersons/PersonsBigramsFormatted/CNBC_2017.csv done: 43.6469